This notebook is used to run some quality control on the individual waveforms and represent this as a number added to the dataframe

In [ ]:
import pandas as pd
import csv
import obspy
import matplotlib.pyplot as plt
import numpy as np
from trace_quality_control import *
import os
import glob

In [ ]:
# Load the catalogue
SEISAN_DATA = os.environ['SEISAN_DATA']
CSVpath = os.path.dirname(SEISAN_DATA)
os.chdir(CSVpath)
list_of_csv_files = sorted(glob.glob('MVOE_catalog*.csv'))
frames = []
for csvfile in list_of_csv_files:
    thisframe = pd.read_csv(csvfile)
    frames.append(thisframe)
mvocat=pd.concat(frames)
mvocat.reset_index()
#mvocat.columns = mvocat.columns.str.lstrip()
mvocat.columns = mvocat.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

write to new CSV file

In [ ]:
mvocat.to_csv('MVOE_CATALOG.csv')

In [ ]:
nrows, ncolumns = mvocat.shape
print(nrows)

Add a quality index to the dataframe

In [ ]:
quality_index = np.zeros(nrows)
mvocat['quality_index'] = quality_index

Loop over every row of the catalog and change quality factor to 1 if trace is good

In [ ]:
for i in range(nrows):
    #if i % 100==0:
    print(i, end = ' ')
    traceid = mvocat.iloc[i].traceid
    wavfile = mvocat.iloc[i].wavfilepath
    if wavfile.empty:
        continue
    try:
        os.path.exists(wavfile)
    except:
        print(wavfile[40:])
    try:
        if not os.path.exists(wavfile):
            continue
    except:
        continue
    st = obspy.read(wavfile).select(id = traceid)
    if not len(st)==1:
        continue
    data = st[0].data
    if check0andMinus1(data):
        quality_index[i] = 1
    else:
        pass
        #print("There is consecutive 0 or -1. in this recording")
print('Done!')
        

In [ ]:
mvocat['quality_index'] = quality_index
mvocat.to_csv('MVOE_CATALOG_QC.csv')

In [ ]:
mvocat.mainclass.value_counts()

In [ ]:
mvocat.subclass.value_counts()

In [ ]:
LVcat = mvocat.loc[mvocat.mainclass == 'LV']
LVcat.subclass.value_counts()

In [ ]:
len(LVcat.wavfilepath.unique())

In [ ]:
import matplotlib.pyplot as plt